In [7]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [19]:
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [20]:
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY is missing in your .env")
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [13]:
chat_llm = ChatGoogleGenerativeAI(model ="gemini-2.5-flash", temperature=0.7)

In [14]:
chat_llm

ChatGoogleGenerativeAI(profile={'max_input_tokens': 1048576, 'max_output_tokens': 65536, 'image_inputs': True, 'audio_inputs': True, 'pdf_inputs': True, 'video_inputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'image_tool_message': True, 'tool_choice': True}, google_api_key=SecretStr('**********'), model='gemini-2.5-flash', client=<google.genai.client.Client object at 0x000002BA44591190>, default_metadata=(), model_kwargs={})

In [21]:
# from google import genai

# client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

# for model in client.models.list():
#     print(model.name)

In [15]:
chat_llm.invoke('Hello, How are you?')

AIMessage(content="Hello! I'm an AI, so I don't experience feelings like humans do, but I'm ready and functioning well.\n\nHow can I help you today?", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019c56aa-bc09-7993-8d9a-c6574dff5aae-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 7, 'output_tokens': 91, 'total_tokens': 98, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 55}})

In [23]:
from typing_extensions import TypedDict, Annotated
import operator

In [24]:
from langchain_core.messages import AnyMessage, HumanMessage,AIMessage, SystemMessage

In [25]:
class GraphState(TypedDict):
    message:Annotated[list[AnyMessage], operator.add]

In [26]:
# ['hai'] ['hai', 'how are you'] like this we can append in the list of messages

In [ ]:
def llm_call(state: GraphState) -> dict:
    """Call the LLM using conversation messages and append AI response."""
    response = chat_llm.invoke(state["messages"])  # AIMessage
    return {
        "messages": [response]
    }

In [28]:
def token_counter(state: GraphState) -> dict:
    """Count tokens (simple word count) in the last AI message."""
    last_msg = state["messages"][-1]
    text = last_msg.content
    token_number = len(text.split())
    summary = f"Total token number in the generated answer (word count) is {token_number}"
    return {
        "messages": [AIMessage(content=summary)]
    }